# Construct a Simple RAG based on Vector Database out of text using a LLM

```bash
! pip install llama-index
! pip install llama-cpp-python
! pip install transformers
! pip install torch


```

In [1]:
####### load chunked data
import re
import json
import os

with open("./chunks.json", "r") as f:
    chunks = json.loads(f.read())
print(len(chunks)) #document chunks are nodes in LlamaIndex

97


In [8]:
chunk = chunks[0]
metadata = chunk['kwargs']['metadata']
content = chunk['kwargs']['page_content']
print(metadata)
print(content)
print("="*80)
chunk = chunks[1]
metadata = chunk['kwargs']['metadata']
content = chunk['kwargs']['page_content']
print(metadata)
print(content)


{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}
Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a 

In [11]:
####### Build Nodes from chunked data
# A Node represents a “chunk” of a source Document, whether that is a text chunk, an image, or other. 
# Similar to Documents, they contain metadata and relationship information with other nodes.
from llama_index.schema import TextNode
nodes = []
for chunk in chunks:
    metadata = chunk['kwargs']['metadata']
    content = chunk['kwargs']['page_content']
    #content = re.sub('[^A-Z.a-z]+', ' ', content)
    nodes.append(TextNode(text=content, id_=metadata['uuid'], metadata=metadata))
print(len(nodes))
print(nodes[0])
print(nodes[0].metadata)

97
Node ID: 0af897c6-3391-496e-a93c-a5a18b350f50
Text: Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14
March 1879 – 18 April 1955) was a German-born theoretical physicist
who is widely held to be one of the greatest and most influential
scientists of all time. Best known for developing the theory of
relativity, Einstein also made important contributions to quantum
mechanics, and was ...
{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}


In [12]:
######### Load LLM
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
import os

#model = "llama-2-13b-chat.Q5_K_M.gguf"
model = "mistral-7b-openorca.Q5_K_M.gguf" #https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF
llama_cpp_path = "/Users/done/Documents/dneumnn/llama.cpp/models"
model_path = os.path.join(llama_cpp_path, model)

llm = LlamaCPP(
    model_url=None,
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=2000,
    context_window=4000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/done/Documents/dneumnn/llama.cpp/models/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weig

In [36]:
######### test model
response = llm.complete("Q: Please tellme the relationship between Albert Einstein and Elsa Löwenthal and her profession. A:", temperature=0)
print(response.text)

Llama.generate: prefix-match hit




Albert Einstein was a renowned physicist, best known for his theory of relativity. Elsa Löwenthal was his second wife. She was a social worker and psychotherapist by profession. The relationship between Albert Einstein and Elsa Löwenthal was that of husband and wife.



llama_print_timings:        load time =     471.35 ms
llama_print_timings:      sample time =       6.29 ms /    68 runs   (    0.09 ms per token, 10815.97 tokens per second)
llama_print_timings: prompt eval time =     315.00 ms /    10 tokens (   31.50 ms per token,    31.75 tokens per second)
llama_print_timings:        eval time =    1606.88 ms /    67 runs   (   23.98 ms per token,    41.70 tokens per second)
llama_print_timings:       total time =    2006.92 ms


In [14]:
# use Huggingface embeddings
# https://huggingface.co/BAAI/bge-small-en-v1.5
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# create a service context
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)
from llama_index import VectorStoreIndex
index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)

# Persist the index to disk
storage_directory = "./"
index.storage_context.persist(persist_dir=storage_directory)

Generating embeddings:   0%|          | 0/97 [00:00<?, ?it/s]

In [15]:
with open("./index_store.json", "r") as f:
    data = json.loads(f.read())    

In [22]:
for i, x in enumerate(data['index_store/data']['d32b0835-f4ff-499f-a892-b43265433b27']['__data__']):
    print(i, type(x))

0 <class 'str'>
1 <class 'str'>
2 <class 'str'>
3 <class 'str'>
4 <class 'str'>
5 <class 'str'>
6 <class 'str'>
7 <class 'str'>
8 <class 'str'>
9 <class 'str'>
10 <class 'str'>
11 <class 'str'>
12 <class 'str'>
13 <class 'str'>
14 <class 'str'>
15 <class 'str'>
16 <class 'str'>
17 <class 'str'>
18 <class 'str'>
19 <class 'str'>
20 <class 'str'>
21 <class 'str'>
22 <class 'str'>
23 <class 'str'>
24 <class 'str'>
25 <class 'str'>
26 <class 'str'>
27 <class 'str'>
28 <class 'str'>
29 <class 'str'>
30 <class 'str'>
31 <class 'str'>
32 <class 'str'>
33 <class 'str'>
34 <class 'str'>
35 <class 'str'>
36 <class 'str'>
37 <class 'str'>
38 <class 'str'>
39 <class 'str'>
40 <class 'str'>
41 <class 'str'>
42 <class 'str'>
43 <class 'str'>
44 <class 'str'>
45 <class 'str'>
46 <class 'str'>
47 <class 'str'>
48 <class 'str'>
49 <class 'str'>
50 <class 'str'>
51 <class 'str'>
52 <class 'str'>
53 <class 'str'>
54 <class 'str'>
55 <class 'str'>
56 <class 'str'>
57 <class 'str'>
58 <class 'str'>
59 <cla

In [23]:
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)


In [37]:
response = query_engine.query("Q: Please tellme the relationship between Albert Einstein and Elsa Löwenthal and her profession. A:")

Llama.generate: prefix-match hit


['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_formatted_sources', 'metadata', 'response', 'source_nodes']



llama_print_timings:        load time =     471.35 ms
llama_print_timings:      sample time =       2.06 ms /    20 runs   (    0.10 ms per token,  9694.62 tokens per second)
llama_print_timings: prompt eval time =    3562.48 ms /  1349 tokens (    2.64 ms per token,   378.67 tokens per second)
llama_print_timings:        eval time =     513.10 ms /    19 runs   (   27.01 ms per token,    37.03 tokens per second)
llama_print_timings:       total time =    4103.64 ms


In [30]:
chunks_as_dict = {chunk['kwargs']['metadata']['uuid']: chunk['kwargs']['page_content']  for chunk in chunks}

In [38]:
metadatas = response.metadata
for metadata in metadatas:
    print(metadatas[metadata])
    print(chunks_as_dict[metadata])

{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '9d7da357-c4b5-4bed-8db4-f0440804b5e6'}
part of the divorce settlement, Einstein agreed that if he were to win a Nobel Prize, he would give the money that he received to Marić; she had to wait only two years before her foresight in extracting this promise from him was rewarded.Einstein married Löwenthal in 1919. In 1923, he began a relationship with a secretary named Betty Neumann, the niece of his close friend Hans Mühsam. Löwenthal nevertheless remained loyal to him, accompanying him when he emigrated to the United States in 1933. In 1935, she was diagnosed with heart and kidney problems. She died in December 1936.A volume of Einstein's letters released by Hebrew University of Jerusalem in 2006 added further names to the catalog of women with whom he was romantically involved. They included Margarete Lebach (a blonde Austrian), Estella Katzenel

In [39]:
print(response)

 Albert Einstein and Elsa Löwenthal were married. Elsa was a secretary.


In [ ]:
##########

In [ ]:
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf").encode
    #AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)
